
# <span style="color:#006E7F">__Introduction to Oxford Nanopore Data Analysis__ <a class="anchor"></span>  

Created by C. Tranchant (DIADE-IRD), J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) and A. Dereeper (PHIM-IRD) - May 2022  SouthGreen training 

Adapted by J. Orjuela (DIADE-IRD), F. Sabot (DIADE-IRD) - Novembre 2022
    
# <span style="color:#006E7F"> TP4. Structural Variants Detection using long reads  </center> </span>

</span>


### <span style="color:#006E7F">__0 - Preparing working environment__ <a class="anchor" id="work"></a></span>  
### <span style="color: #4CACBC;"> Create the working directory

In [14]:
mkdir -p /home/jovyan/work/RESULTS/SV
cd /home/jovyan/work//RESULTS/
ls -l

total 8
drwxr-xr-x 2 jovyan users 4096 Jun 17 09:42 Assemblies
drwxr-xr-x 3 jovyan users 4096 Nov 10 21:34 SYRI


### <span style="color: #4CACBC;"> Download two genomes</span>  

In [2]:
wget https://itrop.ird.fr/sv-training/Assemblies.tar.gz
tar xvf Assemblies.tar.gz && rm Assemblies.tar.gz

--2022-11-10 21:15:38--  https://itrop.ird.fr/sv-training/Assemblies.tar.gz
Resolving itrop.ird.fr (itrop.ird.fr)... 91.203.35.184
Connecting to itrop.ird.fr (itrop.ird.fr)|91.203.35.184|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9150274 (8.7M) [application/x-gzip]
Saving to: ‘Assemblies.tar.gz’

Assemblies.tar.gz   100%[===================>]   8.73M  27.6MB/s    in 0.3s    

2022-11-10 21:15:39 (27.6 MB/s) - ‘Assemblies.tar.gz’ saved [9150274/9150274]

Assemblies/
Assemblies/assembly-stats.txt
Assemblies/A8_assembly.fasta
Assemblies/5417_assembly.fasta


In [15]:
ls -R

.:
Assemblies  SYRI

./Assemblies:
5417_assembly.fasta  A8_assembly.fasta  assembly-stats.txt

./SYRI:
out.delta            out.filtered.delta  plotsr_pos.txt  syri.log  syri.summary
out.filtered.coords  plotsr.pdf          SNPs.bed        syri.out  syri.vcf


## <span style="color:#006E7F">1. nucmer and syri <a class="anchor" id="SVsyri"></a></span>  

### <span style="color: #4CACBC;"> Initialization of two variables "genome"</span>  


In [3]:
dir_genome="/home/jovyan/work/SV/"
reference_assembly=$dir_genome"Assemblies/A8_assembly.fasta"
query_assembly=$dir_genome"Assemblies/5417_assembly.fasta"

### <span style="color: #4CACBC;"> Create the working directory for SYRI analysis</span>  

In [7]:
dir_SV=$dir_genome"SYRI/"
mkdir -p $dir_SV
cd $dir_SV

### <span style="color: #4CACBC;"> Aligning genomes using `Nucmer` <a class="anchor" id="nucmer"></a></span>  

#### Alignement of the two genomes

In [6]:
nucmer --maxmatch $reference_assembly $query_assembly

1: PREPARING DATA
2,3: RUNNING mummer AND CREATING CLUSTERS
# reading input file "out.ntref" of length 15306200
# construct suffix tree for sequence of length 15306200
# (maximum reference length is 536870908)
# (maximum query length is 4294967295)
# process 153062 characters per dot
#...................................................................................................
# CONSTRUCTIONTIME /usr/bin/mummer out.ntref 9.02
# reading input file "/home/jovyan/work/SV/Assemblies/5417_assembly.fasta" of length 15276944
# matching query-file "/home/jovyan/work/SV/Assemblies/5417_assembly.fasta"
# against subject-file "out.ntref"
# COMPLETETIME /usr/bin/mummer out.ntref 32.23
# SPACE /usr/bin/mummer out.ntref 29.56
4: FINISHING DATA


In [8]:
ls -lrt

total 1788
-rw-r--r-- 1 jovyan users 1829434 Nov 10 21:18 out.delta


#### Filtering nucmer results

Remove small and lower quality alignments


In [9]:
delta-filter -m -i 90 -l 100 out.delta > out.filtered.delta

In [10]:
ls -lrt

total 2440
-rw-r--r-- 1 jovyan users 1829434 Nov 10 21:18 out.delta
-rw-r--r-- 1 jovyan users  667043 Nov 10 21:18 out.filtered.delta


In [11]:
show-coords -THrd out.filtered.delta > out.filtered.coords

In [12]:
ls -lrt

total 2540
-rw-r--r-- 1 jovyan users 1829434 Nov 10 21:18 out.delta
-rw-r--r-- 1 jovyan users  667043 Nov 10 21:18 out.filtered.delta
-rw-r--r-- 1 jovyan users  100931 Nov 10 21:19 out.filtered.coords


### <span style="color: #4CACBC;"> SV calling using syri <a class="anchor" id="siri"></a></span>  

In [13]:
syri -c out.filtered.coords -d out.filtered.delta -r $reference_assembly -q $query_assembly

/opt/conda/envs/syri_env/lib/python3.9/multiprocessing/pool.py:48: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return list(map(*args))
/opt/conda/envs/syri_env/lib/python3.9/multiprocessing/pool.py:48: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return list(map(*args))
/opt/conda/envs/syri_env/lib/python3.9/multiprocessing/pool.py:48: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-bas

In [15]:
ls

out.delta            out.filtered.delta  syri.out      syri.vcf
out.filtered.coords  syri.log            syri.summary


In [16]:
cat syri.summary

#Structural annotations
#Variation_type	Count	Length_ref	Length_qry
Syntenic regions	197	14079055	14107736
Inversions	32	397553	259052
Translocations	93	91312	90081
Duplications (reference)	152	79812	-
Duplications (query)	161	-	83493
Not aligned (reference)	372	667759	-
Not aligned (query)	413	-	738403


#Sequence annotations
#Variation_type	Count	Length_ref	Length_qry
SNPs	239492	239492	239492
Insertions	31092	-	114051
Deletions	38036	119912	-
Copygains	32	-	13466
Copylosses	28	17778	-
Highly diverged	707	488054	383996
Tandem repeats	23	8965	10360


In [17]:
head syri.vcf

##fileformat=VCFv4.3
##fileDate=20221110
##source=syri
##ALT=<ID=SYN,Description="Syntenic region">
##ALT=<ID=INV,Description="Inversion">
##ALT=<ID=TRANS,Description="Translocation">
##ALT=<ID=INVTR,Description="Inverted Translocation">
##ALT=<ID=DUP,Description="Duplication">
##ALT=<ID=INVDP,Description="Inverted Duplication">
##ALT=<ID=SYNAL,Description="Syntenic alignment">


### <span style="color: #4CACBC;"> Extracting all SNP from syri  <a class="anchor" id="siri"></a></span>  

In [8]:
cat syri.out | grep SNP | cut -f 1,2 > a
cat syri.out | grep SNP | cut -f 4,5 > b
cat syri.out | grep SNP | cut -f 3 > c
awk '{ print $1+1 }' c > d
paste a d c > SNPs.bed
rm a b c d


In [9]:
echo -e $reference_assembly'\t ref' > plotsr_pos.txt
echo -e $query_assembly'\t query' >> plotsr_pos.txt

In [11]:
plotsr --sr syri.out --genomes plotsr_pos.txt -s 500 -o plotsr.pdf -H 8 -W 5

Plotsr - WARNING - Plot plotsr.pdf generated.
Finished
